In [1]:
import numpy as np
import pandas as pd
import jieba
import re
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

## clean data

In [5]:
movie_corpus= pd.read_csv('movie_comments.csv',encoding='utf8')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
movie_corpus.head(2)

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2


In [22]:
XY=movie_corpus.iloc[:,3:].copy()

In [23]:
cleaned_XY=a.dropna(subset=['comment'])

In [24]:
pattern = re.compile('\w+')

In [32]:
def data_clean(string):
    d=''.join(pattern.findall(string))
    s=' '.join(jieba.cut(d))
    return s

In [42]:
a=cleaned_XY['comment']
cleaned_XY['cleaned_comment'] = a.apply(data_clean)

In [45]:
cleaned_XY.drop(labels='comment',axis=1,inplace=True)

In [53]:
cleaned_XY.rename(columns={'star':'Y','cleaned_comment':'X'},inplace=True)

In [54]:
cleaned_XY.head(10)

,Y,X
0,1,吴京 意淫 到 了 脑残 的 地步 看 了 恶心 想 吐
1,2,首映礼 看 的 太 恐怖 了 这个 电影 不讲道理 的 完全 就是 吴京 在 实现 他 这个...
2,2,吴京 的 炒作 水平 不输 冯小刚 但小刚 至少 不会 用 主旋律 来 炒作 吴京 让 人 ...
3,4,凭良心说 好 看到 不像 战狼 1 的 续集 完虐 湄公河 行动
4,1,中二得 很
5,1,犯 我 中华 者 虽远必 诛 吴京 比 这句 话 还要 意淫 一百倍
6,2,脑子 是 个 好 东西 希望 编剧 们 都 能 有
7,4,三星 半 实打实 的 7 分 第一集 在 爱国 主旋律 内部 做 着 各种 置换 与 较劲 ...
8,4,开篇 长镜头 惊险 大气 引人入胜 结合 了 水平 不俗 的 快 剪下 实打实 的 真刀真枪...
9,1,15100 吴京 的 冷峰 在 这部 里 即 像 成龙 又 像杰 森斯坦 森但 体制 外 的...


In [55]:
cleaned_XY.shape

(261495, 2)

In [56]:
cleaned_XY.to_csv('cleaned_for_train.csv')

## train word_vector

In [60]:
with open('cleaned_for_train.csv','r',encoding='utf8') as f:
    word_model = Word2Vec(LineSentence(f),size=100)

In [68]:
word_model.save('news_vector')

In [73]:
a = Word2Vec.load('news_vector')

In [74]:
a=word_model.wv
a.most_similar('我')

[('你', 0.6124907732009888),
 ('俺', 0.5461215972900391),
 ('我会', 0.536949872970581),
 ('大家', 0.5187957286834717),
 ('她', 0.5101457834243774),
 ('当初', 0.5062898397445679),
 ('人', 0.4970146417617798),
 ('观众', 0.4938845634460449),
 ('一直', 0.4937843382358551),
 ('它', 0.48911339044570923)]

## sentence vector

In [78]:
line=jieba.cut('今天天气很好我们一起去郊游吧')

In [82]:
cleaned_XY.rename(columns={'X':'Sen'},inplace=True)

In [83]:
cleaned_XY.head(2)

,Y,Sen
0,1,吴京 意淫 到 了 脑残 的 地步 看 了 恶心 想 吐
1,2,首映礼 看 的 太 恐怖 了 这个 电影 不讲道理 的 完全 就是 吴京 在 实现 他 这个...


In [86]:
model = Word2Vec.load('news_vector')
wv = model.wv

In [108]:
def get_sentence_vector(sentence):
    ws = sentence.split()
    ws = [ a for a in ws if a in wv.vocab]
    matrix = np.sum(np.asanyarray([wv[a] for a in ws ]),axis=0)/ len(ws)
    return matrix

In [110]:
cleaned_XY['X'] =cleaned_XY['Sen'].apply(get_sentence_vector)

In [173]:
cleaned_XY.shape

(261495, 3)

In [200]:
cleaned_XY.dropna(subset=['X'],inplace=True) #remove na

In [205]:
X = np.concatenate([a.reshape(1,-1) for a in cleaned_XY['X']])
Y = np.asarray(cleaned_XY['Y'])

In [202]:
X.shape

(253677, 100)

In [206]:
Y.shape

(253677,)

## data split

In [423]:
X.dtype

dtype('float32')

In [398]:
Y=Y.astype('float32') #type convert

In [399]:
Y.dtype

dtype('float32')

In [400]:
X.dtype

dtype('float32')

In [401]:
data = np.concatenate([X, Y.reshape(-1,1)],axis=1)

In [402]:
data.shape

(253677, 101)

In [403]:
data.dtype

dtype('float32')

In [405]:
data[0]

array([-0.75965554,  0.585755  ,  0.653013  ,  0.5308875 , -0.8291249 ,
       -0.1277238 , -0.26111695,  0.42433444,  0.01836779, -0.43668666,
       -0.25576475, -0.18091045,  0.2649455 , -0.49695602,  1.291426  ,
       -0.78607655, -0.36882496, -0.45865402, -0.82139283,  0.1488651 ,
        0.08990908,  0.45723435,  0.07786971,  0.23718983,  0.5163208 ,
       -0.04585673,  0.69643956,  0.1286376 ,  0.23877235, -0.03645267,
        0.43516123, -0.04573588,  0.06817112, -0.54541665,  0.10132736,
       -0.10005027,  0.4477826 , -0.16605018, -0.05341306,  0.01387484,
       -0.2994766 , -0.01277458,  0.7380431 ,  0.4772904 ,  0.18722458,
       -0.01419878,  0.5047675 ,  0.30156338,  0.5422192 ,  0.03767951,
        0.788621  , -0.0073684 ,  0.20435174,  0.6590789 , -0.7590256 ,
       -0.42855465,  0.6293128 , -0.8665336 , -0.24041514,  0.2800518 ,
       -0.84311867,  0.6918037 ,  0.17803045,  0.55546576,  0.31053945,
       -0.66440886,  0.03638462, -0.36203524, -0.02935234,  0.40

In [406]:
np.random.shuffle(data) #shuffer

In [407]:
n_train=np.floor(data.shape[0] * 0.6).astype('int')
n_valid=np.floor(data.shape[0] * 0.2).astype('int')
n_test=np.floor(data.shape[0] - n_train - n_valid).astype('int')

In [408]:
data.shape

(253677, 101)

In [409]:
def get_y_vector(y,c):
    r = np.zeros([len(y),c])
    for i, j in enumerate(y): r[i,j.astype('int') -1]=1
    return r

In [410]:
train_x = data[:n_train,:-1]
train_y = get_y_vector(data[:n_train,-1],5)
valid_x = data[:n_valid,:-1]
valid_y = get_y_vector(data[:n_valid,-1],5) 
test_x = data[:n_test,:-1]
test_y = get_y_vector(data[:n_test,-1],5)  

In [411]:
train_x.shape

(152206, 100)

In [412]:
valid_x.dtype

dtype('float32')

In [413]:
train_y.shape

(152206, 5)

## train network

In [304]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as bb

In [391]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [419]:
num_nodes= 1024
batch_size = 128
beta = 0.01
num_labels=5

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 100))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_x)
    tf_test_dataset = tf.constant(test_x)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([100, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    relu_layer_dropout = tf.nn.dropout(relu_layer, keep_prob)
    
    logits_2 = tf.matmul(relu_layer_dropout, weights_2) + biases_2
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels))
    # Loss function with L2 Regularization with beta=0.01
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for validation 
    logits_1 = tf.matmul(valid_x, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for test
    logits_1 = tf.matmul(test_x, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction =  tf.nn.softmax(logits_2)

In [422]:
num_steps = 8000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_x[offset:(offset + batch_size), :]
        batch_labels = train_y[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_y)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_y)))

Initialized
Minibatch loss at step 0: 579.12353515625
Minibatch accuracy: 16.4
Validation accuracy: 31.9
Minibatch loss at step 500: 1.5198553800582886
Minibatch accuracy: 32.0
Validation accuracy: 25.2
Minibatch loss at step 1000: 1.513419270515442
Minibatch accuracy: 32.0
Validation accuracy: 31.9
Minibatch loss at step 1500: 1.5888187885284424
Minibatch accuracy: 26.6
Validation accuracy: 22.4
Minibatch loss at step 2000: 1.4975383281707764
Minibatch accuracy: 32.0
Validation accuracy: 25.2
Minibatch loss at step 2500: 1.5519797801971436
Minibatch accuracy: 28.9
Validation accuracy: 31.9
Minibatch loss at step 3000: 1.5060491561889648
Minibatch accuracy: 31.2
Validation accuracy: 25.2
Minibatch loss at step 3500: 1.5578304529190063
Minibatch accuracy: 29.7
Validation accuracy: 31.9
Minibatch loss at step 4000: 1.5184588432312012
Minibatch accuracy: 42.2
Validation accuracy: 31.9
Minibatch loss at step 4500: 1.552945613861084
Minibatch accuracy: 18.0
Validation accuracy: 31.9
Minibat

## test tfidf vector

In [424]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [426]:
movie_corpus['comment'].head(2)

0                                   吴京意淫到了脑残的地步，看了恶心想吐
1    首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...
Name: comment, dtype: object

In [448]:
tfidf_v = TfidfVectorizer(max_features=2000)

In [435]:
td_doc = movie_corpus.copy()

In [436]:
td_doc.head(2)

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2


In [438]:
td_doc.dropna(subset=['comment','star'],axis=0,inplace=True)

In [449]:
a=tfidf_v.fit_transform(td_doc['comment'])

In [450]:
tfidf_array=a.toarray()

In [451]:
tfidf_array.shape

(261495, 2000)

In [498]:
td_doc['star'][568]=2

In [507]:
def get_y_vector1(y,c):
    r = np.zeros([len(y),c])
    for i, j in enumerate(y): r[i,j -1]=1
    return r

In [508]:
X = tfidf_array
Y = get_y_vector1(td_doc['star'].astype('int'),5)

In [509]:
X.shape

(261495, 2000)

In [510]:
Y.shape

(261495, 5)

In [512]:
num_nodes= 2000
batch_size = 128
beta = 0.01
num_labels=5

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 2000))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_x)
    tf_test_dataset = tf.constant(test_x)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([2000, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    relu_layer_dropout = tf.nn.dropout(relu_layer, keep_prob)
    
    logits_2 = tf.matmul(relu_layer_dropout, weights_2) + biases_2
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels))
    # Loss function with L2 Regularization with beta=0.01
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_2)
    
#     # Predictions for validation 
#     logits_1 = tf.matmul(valid_x, weights_1) + biases_1
#     relu_layer= tf.nn.relu(logits_1)
#     logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
#     valid_prediction = tf.nn.softmax(logits_2)
    
#     # Predictions for test
#     logits_1 = tf.matmul(test_x, weights_1) + biases_1
#     relu_layer= tf.nn.relu(logits_1)
#     logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
#     test_prediction =  tf.nn.softmax(logits_2)

In [514]:
num_steps = 3000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (Y.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = X[offset:(offset + batch_size), :]
        batch_labels = Y[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
#             print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_y)))
#     print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_y)))

Initialized
Minibatch loss at step 0: 15524.1220703125
Minibatch accuracy: 22.7
Minibatch loss at step 500: 1.550822138786316
Minibatch accuracy: 38.3
Minibatch loss at step 1000: 1.4068701267242432
Minibatch accuracy: 24.2
Minibatch loss at step 1500: 1.38636314868927
Minibatch accuracy: 27.3
Minibatch loss at step 2000: 1.3549168109893799
Minibatch accuracy: 49.2
Minibatch loss at step 2500: 1.4373871088027954
Minibatch accuracy: 35.9
